# Introduction

As a data analyst tasked to analize the datasets provided (Indian Start-up Funding 2018-2021) and advice which sector is lucrative enough to invest into based on findings and conclusions drawn from the analysis. 

In [1]:
## Load appropriate modules
import pandas as pd


Load all datasets

In [5]:
df_2018 = pd.read_csv("./India Startup Funding/startup_funding2018.csv")
df_2019 = pd.read_csv("./India Startup Funding/startup_funding2019.csv")
df_2020 = pd.read_csv("./India Startup Funding/startup_funding2020.csv")
df_2021 = pd.read_csv("./India Startup Funding/startup_funding2021.csv")